<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: Ratas.py 🐁

- Nombre de alumno 1: Geraldyn Pérez
- Nombre de alumno 2: Diego Rojas


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Geral37/MDS7202.git)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [2]:
# Inserte su código aquí
df_retail = pd.read_pickle('online_retail_data.pickle')
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,ID489434,79323P,PINK CHERRY LIGHTS,12.0,NaT,6.75,13085.0,United Kingdom
2,ID489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [ ]:
def explore_data(dataframe_in, columns):
    # Generar histogramas para las columnas especificadas
    for column in columns:
        fig = px.histogram(
            dataframe_in,
            x=column,
            title=f"Histograma de la variable {column}",
            marginal="box"
        )
        fig.show()

    # Conteo de nulos
    nulos = dataframe_in.isnull().sum()
    print("Conteo de datos nulos por variable:")
    print(nulos)

explore_data(df_retail, ['Price', 'Quantity'])

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [4]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, lambda_usuario):
    #Completar código aquí
    self.lambda_usuario = lambda_usuario
    self.q1 = None
    self.q3 = None
    self.ric = None

  def fit(self, X):
    #Completar código aquí
    self.q1 = X.quantile(0.25)
    self.q3 = X.quantile(0.75)
    self.ric = self.q3 - self.q1

    return self

  def transform(self, X):
    #Completar código aquí
    lower_limit = self.q1 - self.lambda_usuario * self.ric
    upper_limit = self.q3 + self.lambda_usuario * self.ric

    mask = (X <= upper_limit) & (X >= lower_limit)
    X_sin_outlier = X[mask.all(axis=1)]

    return X_sin_outlier

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

In [5]:
df_retail2 = IQR(lambda_usuario=1.5).fit_transform(df_retail[['Price', 'Quantity']])
df_retail2

,Price,Quantity
0,6.95,12.0
1,6.75,12.0
2,6.75,12.0
4,1.25,24.0
5,1.65,24.0
...,...,...
400941,1.25,2.0
400942,2.95,2.0
400943,3.75,1.0
400944,3.75,1.0


#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [6]:
df_retail

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,ID489434,79323P,PINK CHERRY LIGHTS,12.0,NaT,6.75,13085.0,United Kingdom
2,ID489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
400942,ID538171,22271,FELTCRAFT DOLL ROSIE,2.0,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
400943,ID538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1.0,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400944,ID538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1.0,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400945,ID538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,NaN,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [7]:
df_retail.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']

categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate',
                       'Customer ID', 'Country']

# Definicion del pipeline
numeric_transformations = Pipeline(steps=[
    ('iqr', IQR(lambda_usuario=1.5))] # Paso para IQR
                                   )

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

print(f"Tamaño original: {len(df_retail)}")
print(f"Tamaño después de aplicar IQR: {len(df_iqr)}")

# Gráficos
explore_data(df_iqr, ['Price', 'Quantity'])

Aumentan los NA en las variables numéricas a las que se les aplica IQR. Ahora, cada gráfico es visible e interpretable de mejor manera.

Al reducir el valor de $λ$, aumenta la cantidad de NA, dado que es una ventana pequeña y considera una menor cantidad de puntos dentro del rango intercuartílico.

Se probó con varios $λ$ y se optó por mantener λ=1.5

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
# Escriba su respuesta aquí

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']

categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate',
                       'Customer ID', 'Country']

# Definición del pipeline para las transformaciones categóricas
categoric_transformations = Pipeline(steps=[
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))  # Paso para imputar valores faltantes con la moda
])

# Definición del pipeline para las transformaciones numéricas
numeric_transformations = Pipeline(steps=[
    ('iqr_outlier_removal', IQR(lambda_usuario=1.5)),  # Eliminación de outliers
    ('mean_imputer', SimpleImputer(missing_values = np.nan, strategy='mean'))  # Imputar valores faltantes con la media
])

# Definición del pipeline para las transformaciones numéricas
numeric_transformations = Pipeline(steps=[
    ('mean_imputer', SimpleImputer(missing_values = np.nan, strategy='mean')),
    ('iqr_outlier_removal', IQR(lambda_usuario=1.5)),  # Eliminación de outliers
      # Imputar valores faltantes con la media
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_mean_imputer = column_transformer.fit_transform(df_retail)

# Gráficos
explore_data(df_mean_imputer, ['Price', 'Quantity'])

*Se reduce la cantidad de NA en las variables numéricas. La distribución de Precio se mantiene visualmente, mientras que en la de Cantidad se observa un desplazamiento de la mediana hacia la derecha.*

In [ ]:
# Escriba su respuesta aquí

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']

#categorical_columns = (...)
categorical_columns = ['Country', 'InvoiceDate']

inutiles = ['Invoice', 'StockCode', 'Description', 'Customer ID']

# Definición del pipeline para las transformaciones categóricas
categoric_transformations = Pipeline(steps=[
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))  # Paso para imputar valores faltantes con la moda
])

# Definición del pipeline para las transformaciones numéricas
numeric_transformations = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=3)),  # Paso para imputar valores faltantes con KNN
    ('iqr_outlier_removal', IQR(lambda_usuario=1.5))  # Paso de eliminación de outliers
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns),
                                         ('pasar', 'passthrough', inutiles)
                                        ],
                                        verbose_feature_names_out = False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_knn_imputer = column_transformer.fit_transform(df_retail)

# Gráficos
explore_data(df_knn_imputer, ['Price', 'Quantity'])

*Deja una cantidad más baja de NA. La distribución de Precio se mantiene.*

*Por otra parte, la distribución de Cantidad también se desplaza a la derecha.*

*Al comparar entre la imputación por media y KNN, se observa que dejan una cantidad de NA diferente, aunque cercana, y las variables numéricas distribuyen visualmente igual.*

*Sería posible mejorar el resultado variando la cantidad de vecinos, pero por tiempos de ejecución, aunque se probó con más valores, se muestra finalmente para 3.*

*Así, el método mejor método debería ser KNN con una cantidad de vecinos no determinada, pero para que se pueda correr y comparar resultados, se elige la imputación por el promedio.*

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [32]:
df_retail

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,ID489434,79323P,PINK CHERRY LIGHTS,12.0,NaT,6.75,13085.0,United Kingdom
2,ID489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
400942,ID538171,22271,FELTCRAFT DOLL ROSIE,2.0,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
400943,ID538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1.0,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400944,ID538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1.0,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400945,ID538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,NaN,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [33]:
def custom_features(dataframe_in):
  df = dataframe_in.copy()
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
  df.sort_values(by=['Customer ID', 'InvoiceDate'], inplace=True)
  #df['Length'] = df.groupby('Customer ID')['InvoiceDate'].diff().dt.days
  # Agrupar por 'Customer ID' y calcular la primera y última fecha de compra
  LRMFP = df.groupby('Customer ID')['InvoiceDate'].agg(['min', 'max']).reset_index()
  # Calcular la diferencia en días entre la última y la primera compra
  LRMFP['Length'] = (LRMFP['max'] - LRMFP['min']).dt.days
  L = LRMFP[['Customer ID', 'Length']]

  hoy = df.InvoiceDate.max()
  LRMFP['Recency'] = (hoy - LRMFP['max']).dt.days

  gasto_diario = df.groupby(['Customer ID', 'InvoiceDate'])['Price'].sum().reset_index()
  M = gasto_diario.groupby('Customer ID')['Price'].mean().reset_index()
  M.columns = ['Customer ID', 'Monetary']

  LRMFP = pd.merge(LRMFP, M, on='Customer ID')

  F = df.groupby('Customer ID')['InvoiceDate'].nunique().reset_index()
  F.columns = ['Customer ID', 'Frequency']
  LRMFP = pd.merge(LRMFP, F, on='Customer ID')

  # Eliminar duplicados de fechas para contar una única visita por día
  visitas_unicas = df[['Customer ID', 'InvoiceDate']].drop_duplicates()

  # Calcular la periodicidad (desviación estándar de días entre visitas únicas)
  visitas_unicas['dif_dias'] = visitas_unicas.groupby('Customer ID')['InvoiceDate'].transform(lambda x: x.diff().dt.days)
  P = visitas_unicas.groupby('Customer ID')['dif_dias'].std().reset_index()
  P.columns = ['Customer ID', 'Periodicity']
  P['Periodicity'] = P['Periodicity'].fillna(0)  # Asignar 0 si es NaN

  LRMFP = pd.merge(LRMFP, P, on='Customer ID')

  LRMFP = LRMFP.drop(['min', 'max'], axis=1)

  LRMFP[['Length', 'Recency', 'Frequency']] = LRMFP[['Length', 'Recency', 'Frequency']].apply(
    lambda x: x.astype(int) if x.notna().all() else x)

  LRMFP['Monetary'] = LRMFP['Monetary'].round(2)
  LRMFP['Periodicity'] = LRMFP['Periodicity'].round(1)

  return LRMFP

custom_features(df_mean_imputer)

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196,164,17.20,12,33.3
1,12347.0,274,2,47.73,3,141.4
2,12348.0,0,73,14.39,1,0.0
3,12349.0,234,42,53.18,4,75.3
4,12351.0,0,10,36.71,1,0.0
...,...,...,...,...,...,...
4309,18283.0,275,17,52.73,7,59.3
4310,18284.0,0,66,35.59,1,0.0
4311,18285.0,0,295,16.25,1,0.0
4312,18286.0,247,111,46.45,3,59.4


Se utiliza como supuesto que 1 visita corresponde a todos los registros para un Customer ID de un día completo, en lugar de considerar cada fila como una visita.

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
# Escriba su respuesta aquí

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']

categorical_columns = ['InvoiceDate', 'Country']

omitir = ['Invoice', 'StockCode', 'Description', 'Customer ID']

# Definición del pipeline para las transformaciones numéricas
numeric_transformations = Pipeline(steps=[
    ('mean_imputer', SimpleImputer(missing_values = np.nan, strategy='mean')),  # Paso para imputar valores faltantes con el promedio
    ('iqr_outlier_removal', IQR(lambda_usuario=1.5))  # Paso de eliminación de outliers
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns),
                                         ('pasar', 'passthrough', omitir)
                                        ],
                                        verbose_feature_names_out = False)

# Definición del pipeline
retail_pipeline = Pipeline(steps=[
    ('col_transformer', column_transformer),
    ('custom_features', FunctionTransformer(custom_features, validate=False))
])

retail_pipeline.set_output(transform='pandas')

# Aplicar el pipeline
df_custom = retail_pipeline.fit_transform(df_retail)

# Explorar los datos
explore_data(df_custom, ['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])

Length: la mayoría son clientes nuevos y otro grupo tiene cerca de 250 días de antigüedad.

Recency: La mayoría de los clientes visitó hace menos de 50 días.

Monetary: la mayoría hace compras de bajo precio.

Frequency: la mayoría visita de manera poco frecuente.

Periodicity: la mayoría tiene periodo 0 porque visitó solo 1 vez y existe otro grupo con un período de 20 días.

**Insight:** La mayoría de los clientes son nuevos y realizan compras por montos bajos, mientras que existe otro grupo de clientes más antiguos y que no visitan hace mucho tiempo.

In [37]:
df_custom

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196,164,0.01,12,33.3
1,12347.0,274,2,0.01,3,141.4
2,12348.0,0,73,0.00,1,0.0
3,12349.0,234,42,0.01,4,75.3
4,12351.0,0,10,0.00,1,0.0
...,...,...,...,...,...,...
4309,18283.0,275,17,0.02,7,59.3
4310,18284.0,0,66,0.00,1,0.0
4311,18285.0,0,295,0.00,1,0.0
4312,18286.0,247,111,0.01,3,59.4


Listo (Y)

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [38]:
class MinMax(BaseEstimator, TransformerMixin):

    def __init__(self):
      self.min = None
      self.max = None

    def fit(self,X):
        #Agregar código aquí
      self.min = X.min()
      self.max = X.max()

      return self

    def transform(self,X):
        #Agregar código aquí
        X_escaler = (X - self.min) / (self.max - self.min)

        return X_escaler

    def set_output(self,transform='default'):
        #No modificar este método
        return self

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
# Agregar el paso minmax al pipeline numeric_transformations
numeric_transformations = Pipeline(steps=[
    ('iqr_outlier_removal', IQR(lambda_usuario=1.5)),  # Paso de eliminación de outliers
    ('mean_imputer', SimpleImputer(missing_values = np.nan, strategy='mean')),  # Paso para imputar valores faltantes con la media
    ('minmax', MinMax()) # Paso para usar minmax
])

# Defina el dataframe df_minmax
column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
  ], verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

df_minmax = column_transformer.fit_transform(df_retail)

# Usar explore_data en df_retail y en df_minmax
print("Dataframe: df_retail -----------------------")
explore_data(df_retail, ['Price', 'Quantity'])

In [ ]:
print("Dataframe: df_minmax -----------------------")
explore_data(df_minmax, ['Price', 'Quantity'])

Hay una menor cantidad de outliers y la distribución de cada variable es visible, en contraste con la situación inicial donde la mayoría de la información está concentrada en valores bajos y el resto de los puntos distribuidos difusamente. Ahora es posible discriminar e interpretar la información de manera directa.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Hola Mr. Cheems!

Usar pipelines en Machine Learning es muy útil porque nos permite organizar todo el proceso de transformación de datos de manera eficiente y automática. En lugar de aplicar cada transformación manualmente (como limpiar outliers o escalar datos), el pipeline sigue un flujo bien estructurado, asegurando que no cometamos errores y ahorrándonos mucho tiempo.

Además, los pipelines garantizan que los resultados sean reproducibles. Cada vez que entrenamos el modelo, las mismas transformaciones se aplican de manera idéntica, lo que nos ayuda a evitar errores humanos y a mantener consistencia. También nos protegen contra la fuga de información, ya que las transformaciones se aplican en el momento correcto, evitando que el modelo obtenga pistas que no debería tener.

Otra ventaja es que facilitan la experimentación. Si queremos probar diferentes enfoques, podemos modificar el pipeline fácilmente sin desordenar el resto del proceso. Finalmente, integran el entrenamiento del modelo con las transformaciones, asegurando que el modelo siempre reciba los datos en el formato correcto. Todo el flujo es más ordenado y eficiente, como en nuestro pipeline que combina IQR y Min-Max para los datos.*Escriba su respuesta aquí*

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>